In [ ]:
!curl -O image.png --limit-rate 100M --retry 5 --continue-at - https://eoimages.gsfc.nasa.gov/images/imagerecords/73000/73751/world.topo.bathy.200407.3x21600x21600.A2.png


In [ ]:
!wget -O downloaded_image.png https://eoimages.gsfc.nasa.gov/images/imagerecords/73000/73751/world.topo.bathy.200407.3x21600x21600.D1.png

In [ ]:
%%writefile edge.c
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <math.h>
#include <png.h> // Include libpng

// Structure to represent an image
typedef struct {
  uint8_t *data;
  int width;
  int height;
  int color_type;
  int bit_depth;
} Image;

// Function prototypes
void my_png_error(png_structp png_ptr, const char *message);
Image readPngImage(const char *filename);
void freeImage(Image img);
Image createImage(int width, int height);
void writePngImage(const char *filename, Image img);
Image sobelEdgeDetection(Image input);

// Error handling function
void my_png_error(png_structp png_ptr, const char *message) {
  fprintf(stderr, "PNG error: %s\n", message);
  longjmp(png_jmpbuf(png_ptr), 1);
}

// Function to read a PNG image
Image readPngImage(const char *filename) {
  Image img;
  png_structp png_ptr = NULL;
  png_infop info_ptr = NULL;
  FILE *fp = NULL;

  // Open file
  fp = fopen(filename, "rb");
  if (!fp) {
    fprintf(stderr, "Error opening file: %s\n", filename);
    exit(1);
  }

  // Create png struct
  png_ptr = png_create_read_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);
  if (!png_ptr) {
    fprintf(stderr, "Error creating png_struct\n");
    fclose(fp);
    exit(1);
  }

  // Create info struct
  info_ptr = png_create_info_struct(png_ptr);
  if (!info_ptr) {
    fprintf(stderr, "Error creating info_struct\n");
    png_destroy_read_struct(&png_ptr, (png_infopp)NULL, (png_infopp)NULL);
    fclose(fp);
    exit(1);
  }

  // Set error handler
  if (setjmp(png_jmpbuf(png_ptr))) {
    png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
    fclose(fp);
    freeImage(img); // Make sure to free image memory if allocated
    exit(1);
  }
  png_set_error_fn(png_ptr, NULL, my_png_error, NULL);

  // Initialize PNG reading
  png_init_io(png_ptr, fp);
  png_read_info(png_ptr, info_ptr);

  // Get image info
  img.width = png_get_image_width(png_ptr, info_ptr);
  img.height = png_get_image_height(png_ptr, info_ptr);
  img.color_type = png_get_color_type(png_ptr, info_ptr);
  img.bit_depth = png_get_bit_depth(png_ptr, info_ptr);

  // Convert to 8-bit grayscale if necessary
  if (img.bit_depth == 16)
    png_set_strip_16(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_PALETTE)
    png_set_palette_to_rgb(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_GRAY && img.bit_depth < 8)
    png_set_expand_gray_1_2_4_to_8(png_ptr);
  if (png_get_valid(png_ptr, info_ptr, PNG_INFO_tRNS))
    png_set_tRNS_to_alpha(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_RGB ||
      img.color_type == PNG_COLOR_TYPE_RGB_ALPHA ||
      img.color_type == PNG_COLOR_TYPE_PALETTE)
    png_set_rgb_to_gray_fixed(png_ptr, 1, -1, -1);
  if (img.color_type == PNG_COLOR_TYPE_RGBA)
    png_set_strip_alpha(png_ptr);

  png_read_update_info(png_ptr, info_ptr);

  // Allocate memory for image data
  img.data = (uint8_t *)malloc(img.width * img.height * sizeof(uint8_t));
  if (!img.data) {
    fprintf(stderr, "Memory allocation failed\n");
    png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
    fclose(fp);
    exit(1);
  }

  // Read image data
  png_bytep *row_pointers = (png_bytep *)malloc(sizeof(png_bytep) * img.height);
  for (int y = 0; y < img.height; y++) {
    row_pointers[y] = img.data + y * img.width;
  }

  png_read_image(png_ptr, row_pointers);

  // Cleanup
  png_read_end(png_ptr, info_ptr);
  png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
  fclose(fp);
  free(row_pointers);

  return img;
}

// Function to free image memory
void freeImage(Image img) {
  free(img.data);
}

// Function to create an image
Image createImage(int width, int height) {
  Image img;
  img.width = width;
  img.height = height;
  img.data = (uint8_t *)malloc(width * height * sizeof(uint8_t));
  if (img.data == NULL) {
    fprintf(stderr, "Memory allocation failed\n");
    exit(1);
  }
  return img;
}

// Function to apply the Sobel operator for edge detection
Image sobelEdgeDetection(Image input) {
  int width = input.width;
  int height = input.height;

  Image output = createImage(width, height);

  // Sobel kernels
  int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
  int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};

  for (int y = 1; y < height - 1; y++) {
    for (int x = 1; x < width - 1; x++) {
      // Calculate gradient in x and y directions
      int sumX = 0;
      int sumY = 0;

      for (int i = -1; i <= 1; i++) {
        for (int j = -1; j <= 1; j++) {
          sumX += Gx[i + 1][j + 1] *
                  input.data[(y + i) * width + (x + j)];
          sumY += Gy[i + 1][j + 1] *
                  input.data[(y + i) * width + (x + j)];
        }
      }

      // Calculate the magnitude of the gradient
      int gradientMagnitude =
          (int)sqrt(sumX * sumX + sumY * sumY);

      // Clamp the value to the range 0-255
      if (gradientMagnitude < 0) {
        gradientMagnitude = 0;
      } else if (gradientMagnitude > 255) {
        gradientMagnitude = 255;
      }

      output.data[y * width + x] = (uint8_t)gradientMagnitude;
    }
  }

  return output;
}

// Function to write a PNG image from raw image data
void writePngImage(const char *filename, Image img) {
  FILE *fp = fopen(filename, "wb");
  if (!fp) {
    fprintf(stderr, "Error opening file: %s\n", filename);
    exit(1);
  }

  png_structp png_ptr = png_create_write_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);
  if (!png_ptr) {
    fprintf(stderr, "Error creating png_struct\n");
    fclose(fp);
    exit(1);
  }

  png_infop info_ptr = png_create_info_struct(png_ptr);
  if (!info_ptr) {
    fprintf(stderr, "Error creating info_struct\n");
    png_destroy_write_struct(&png_ptr, (png_infopp)NULL);
    fclose(fp);
    exit(1);
  }

  if (setjmp(png_jmpbuf(png_ptr))) {
    png_destroy_write_struct(&png_ptr, &info_ptr);
    fclose(fp);
    exit(1);
  }

  png_set_compression_level(png_ptr, 9);  // Set compression level (optional)

  png_init_io(png_ptr, fp);
  
  png_set_IHDR(
    png_ptr, info_ptr, img.width, img.height,
    8, PNG_COLOR_TYPE_GRAY, PNG_INTERLACE_NONE, PNG_COMPRESSION_TYPE_DEFAULT, PNG_FILTER_TYPE_DEFAULT
  );

  png_write_info(png_ptr, info_ptr);

  png_bytep row = (png_bytep)malloc(img.width * sizeof(uint8_t));
  for (int y = 0; y < img.height; y++) {
    for (int x = 0; x < img.width; x++) {
      row[x] = img.data[y * img.width + x];
    }
    png_write_row(png_ptr, row);
  }

  png_write_end(png_ptr, info_ptr);

  png_destroy_write_struct(&png_ptr, &info_ptr);
  fclose(fp);
  free(row);
}

int main() {
  // Example Usage:
  const char *inputFilename = "downloaded_image.png"; // Input PNG image
  const char *outputFilename = "output.png"; // Output edge-detected image (PNG)

  // Read the input image
  Image inputImage = readPngImage(inputFilename);

  // Apply Sobel edge detection
  Image edgeDetectedImage = sobelEdgeDetection(inputImage);

  // Write the output image as PNG
  writePngImage(outputFilename, edgeDetectedImage);

  // Free allocated memory
  freeImage(inputImage);
  freeImage(edgeDetectedImage);

  printf("Edge detection complete. Output written to %s\n", outputFilename);

  return 0;
}




In [ ]:
!gcc -o edge edge.c -lm -lpng

In [ ]:
!./edge

In [ ]:
%%writefile edge_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <math.h>
#include <png.h>
#include <cuda_runtime.h>
#include <time.h>  // For measuring CPU time

// Structure to represent an image
typedef struct {
    uint8_t *data;
    int width;
    int height;
    int color_type;
    int bit_depth;
} Image;

void freeImage(Image);
void my_png_error(png_structp png_ptr, const char *message);
void my_png_error(png_structp png_ptr, const char *message) {
  fprintf(stderr, "PNG error: %s\n", message);
  longjmp(png_jmpbuf(png_ptr), 1);
}

// CUDA kernel for Sobel edge detection
__global__ void sobelKernel(uint8_t *input, uint8_t *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // Ensure thread is within image bounds
    if (x >= 1 && x < width - 1 && y >= 1 && y < height - 1) {
        int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
        int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};

        int sumX = 0;
        int sumY = 0;

        // Apply Sobel operator
        for (int i = -1; i <= 1; i++) {
            for (int j = -1; j <= 1; j++) {
                sumX += Gx[i + 1][j + 1] * input[(y + i) * width + (x + j)];
                sumY += Gy[i + 1][j + 1] * input[(y + i) * width + (x + j)];
            }
        }

        // Calculate gradient magnitude
        int gradientMagnitude = (int)sqrtf(sumX * sumX + sumY * sumY);

        // Clamp the value to the range 0-255
        gradientMagnitude = min(max(gradientMagnitude, 0), 255);

        // Write to output
        output[y * width + x] = (uint8_t)gradientMagnitude;
    }
}

// Function to read a PNG image
Image readPngImage(const char *filename) {
  Image img = {NULL, 0, 0, 0, 0};  // Initialize the image structure
  png_structp png_ptr = NULL;
  png_infop info_ptr = NULL;
  FILE *fp = NULL;

  // Open file
  fp = fopen(filename, "rb");
  if (!fp) {
    fprintf(stderr, "Error opening file: %s\n", filename);
    exit(1);
  }

  // Create png struct
  png_ptr = png_create_read_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);
  if (!png_ptr) {
    fprintf(stderr, "Error creating png_struct\n");
    fclose(fp);
    exit(1);
  }

  // Create info struct
  info_ptr = png_create_info_struct(png_ptr);
  if (!info_ptr) {
    fprintf(stderr, "Error creating info_struct\n");
    png_destroy_read_struct(&png_ptr, (png_infopp)NULL, (png_infopp)NULL);
    fclose(fp);
    exit(1);
  }

  // Set error handler
  if (setjmp(png_jmpbuf(png_ptr))) {
    png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
    fclose(fp);
    if (img.data) {
        freeImage(img); // Free memory only if it was allocated
    }
    exit(1);
  }
  png_set_error_fn(png_ptr, NULL, my_png_error, NULL);

  // Initialize PNG reading
  png_init_io(png_ptr, fp);
  png_read_info(png_ptr, info_ptr);

  // Get image info
  img.width = png_get_image_width(png_ptr, info_ptr);
  img.height = png_get_image_height(png_ptr, info_ptr);
  img.color_type = png_get_color_type(png_ptr, info_ptr);
  img.bit_depth = png_get_bit_depth(png_ptr, info_ptr);

  // Convert to 8-bit grayscale if necessary
  if (img.bit_depth == 16)
    png_set_strip_16(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_PALETTE)
    png_set_palette_to_rgb(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_GRAY && img.bit_depth < 8)
    png_set_expand_gray_1_2_4_to_8(png_ptr);
  if (png_get_valid(png_ptr, info_ptr, PNG_INFO_tRNS))
    png_set_tRNS_to_alpha(png_ptr);
  if (img.color_type == PNG_COLOR_TYPE_RGB ||
      img.color_type == PNG_COLOR_TYPE_RGB_ALPHA ||
      img.color_type == PNG_COLOR_TYPE_PALETTE)
    png_set_rgb_to_gray_fixed(png_ptr, 1, -1, -1);
  if (img.color_type == PNG_COLOR_TYPE_RGBA)
    png_set_strip_alpha(png_ptr);

  png_read_update_info(png_ptr, info_ptr);

  // Allocate memory for image data
  img.data = (uint8_t *)malloc(img.width * img.height * sizeof(uint8_t));
  if (!img.data) {
    fprintf(stderr, "Memory allocation failed\n");
    png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
    fclose(fp);
    exit(1);
  }

  // Read image data
  png_bytep *row_pointers = (png_bytep *)malloc(sizeof(png_bytep) * img.height);
  for (int y = 0; y < img.height; y++) {
    row_pointers[y] = img.data + y * img.width;
  }

  png_read_image(png_ptr, row_pointers);

  // Cleanup
  png_read_end(png_ptr, info_ptr);
  png_destroy_read_struct(&png_ptr, &info_ptr, NULL);
  fclose(fp);
  free(row_pointers);

  return img;
}

void writePngImage(const char *filename, Image img) {
  FILE *fp = fopen(filename, "wb");
  if (!fp) {
    fprintf(stderr, "Error opening file: %s\n", filename);
    exit(1);
  }

  png_structp png_ptr = png_create_write_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);
  if (!png_ptr) {
    fprintf(stderr, "Error creating png_struct\n");
    fclose(fp);
    exit(1);
  }

  png_infop info_ptr = png_create_info_struct(png_ptr);
  if (!info_ptr) {
    fprintf(stderr, "Error creating info_struct\n");
    png_destroy_write_struct(&png_ptr, (png_infopp)NULL);
    fclose(fp);
    exit(1);
  }

  if (setjmp(png_jmpbuf(png_ptr))) {
    png_destroy_write_struct(&png_ptr, &info_ptr);
    fclose(fp);
    exit(1);
  }

  png_set_compression_level(png_ptr, 9);  // Set compression level (optional)

  png_init_io(png_ptr, fp);
  
  png_set_IHDR(
    png_ptr, info_ptr, img.width, img.height,
    8, PNG_COLOR_TYPE_GRAY, PNG_INTERLACE_NONE, PNG_COMPRESSION_TYPE_DEFAULT, PNG_FILTER_TYPE_DEFAULT
  );

  png_write_info(png_ptr, info_ptr);

  png_bytep row = (png_bytep)malloc(img.width * sizeof(uint8_t));
  for (int y = 0; y < img.height; y++) {
    for (int x = 0; x < img.width; x++) {
      row[x] = img.data[y * img.width + x];
    }
    png_write_row(png_ptr, row);
  }

  png_write_end(png_ptr, info_ptr);

  png_destroy_write_struct(&png_ptr, &info_ptr);
  fclose(fp);
  free(row);
}

// Function to free image memory
void freeImage(Image img) {
  free(img.data);
}

// Host function to process the image using CUDA
void processImageWithCUDA(const char *inputFilename, const char *outputFilename) {
    // Start measuring time on the host
    clock_t startClock = clock();

    Image inputImage = readPngImage(inputFilename);

    // Allocate device memory
    uint8_t *d_input, *d_output;
    size_t imgSize = inputImage.width * inputImage.height * sizeof(uint8_t);
    cudaMalloc((void **)&d_input, imgSize);
    cudaMalloc((void **)&d_output, imgSize);

    // Copy image data to device
    cudaMemcpy(d_input, inputImage.data, imgSize, cudaMemcpyHostToDevice);

    // Start CUDA events for kernel timing
    cudaEvent_t startEvent, stopEvent;
    float elapsedTime;

    cudaEventCreate(&startEvent);
    cudaEventCreate(&stopEvent);
    cudaEventRecord(startEvent, 0);

    // Define block and grid size
    dim3 blockDim(16, 16);
    dim3 gridDim((inputImage.width + blockDim.x - 1) / blockDim.x,
                 (inputImage.height + blockDim.y - 1) / blockDim.y);

    // Launch CUDA kernel for Sobel edge detection
    sobelKernel<<<gridDim, blockDim>>>(d_input, d_output, inputImage.width, inputImage.height);

    // Check for CUDA errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA error: %s\n", cudaGetErrorString(err));
        exit(1);
    }

    // Record stop event
    cudaEventRecord(stopEvent, 0);
    cudaEventSynchronize(stopEvent);
    cudaEventElapsedTime(&elapsedTime, startEvent, stopEvent);

    // Copy the result back to host
    uint8_t *h_output = (uint8_t *)malloc(imgSize);
    cudaMemcpy(h_output, d_output, imgSize, cudaMemcpyDeviceToHost);

    // Prepare output image structure
    Image edgeDetectedImage;
    edgeDetectedImage.data = h_output;
    edgeDetectedImage.width = inputImage.width;
    edgeDetectedImage.height = inputImage.height;

    // Write the output image as PNG
    writePngImage(outputFilename, edgeDetectedImage);

    // Cleanup
    freeImage(inputImage);
    freeImage(edgeDetectedImage);
    cudaFree(d_input);
    cudaFree(d_output);

    // End measuring time on the host
    clock_t endClock = clock();
    double cpuTime = (double)(endClock - startClock) / CLOCKS_PER_SEC;

    printf("Edge detection complete. Output written to %s\n", outputFilename);
    printf("Processing time (CUDA): %.2f ms\n", elapsedTime);  // CUDA kernel time
    printf("Processing time (CPU): %.2f seconds\n", cpuTime);   // Host processing time
}

int main() {
    // Example usage
    const char *inputFilename = "downloaded_image.png";  // Input PNG image
    const char *outputFilename = "output_cuda.png";           // Output edge-detected image (PNG)

    // Process the image with CUDA
    processImageWithCUDA(inputFilename, outputFilename);

    return 0;
}


In [ ]:
!nvcc edge_gpu.cu -o ec -lm -lpng

In [ ]:
!./ec

In [ ]:
from IPython.display import Image

# Display output_cpu.png
Image(filename='output_cpu.png')

# Display output_cuda.png
Image(filename='output_cuda.png')


In [ ]:
!ls -l